#### Names of people in the group

Please write the names of the people in your group in the next cell.

Frederick Nilsen

In [0]:
# Loading modules that we need
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Add your imports below this line
import itertools # to flatten multidimensional lists
from typing import Tuple

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
posts_df = load_df("/user/hive/warehouse/posts")

# Uncomment if you need
# comments_df = load_df("/user/hive/warehouse/comments")
# badges_df = load_df("/user/hive/warehouse/badges")

#### The problem: mining the interests of experts

The primary role of a questions and answering platform such as Stack Exchange is to connect two types of people. Namely, people who have questions in areas such as computer science or data science and knowledgeable people who can answer those questions reliably. Let's call the first category of people' knowledge seekers' and the second one 'expert users' or 'experts' for short.

Here we want to answer a question related to the diversity of topics that experts are interested in using our data. We want to know if expert users only answer questions in a specific set of topics or their interests include a wide variety of topics.

To answer the above question, we will compute the correlation between a user's expertise level and the diversity of topics of questions they have answered. The first step is to define two variables (or measures); first for 'user expertise level' and then for 'user interest diversity'. Then we will use the Pearson correlation coefficient to measure the linear correlation between the two variables. We define the variables as:

   - VariableA (the measure of user expertise level). We will use the 'Reputation' column from 'users' table, which according to Stack Exchange's documentation "is a rough measurement of how much the community trusts you; it is earned by convincing your peers that you know what you're talking about" as an indicator of a user's expertise level on the platform. 

   - VariableB (The measure of user interest diversity). We measure the diversity of a user's interests by computing the total number of distinct tags associated with the questions each user has answered divided by the total number of unique tags which is 638.

Compute the Pearson correlation coefficient between VariableA and VariableB, and based on the result you've got, answer the following question: 

     Do expert users have specif interests or do they have general interests?

Please explain your reasoning on how you reached your answer.

You should use Apache Spark API for your implementation. You can use the Spark implementation of the Pearson correlation coefficient.

**Implementation plan**

In order to get the correlation between two variables, it is beneficial to have a method for computing them. For the case of reputation, we already have the final expression stored in our data (User's reputation). For the case of interest diversity, we need to use transformations on the existing table information before we can compute the correlation. The interest diversity is measured for each user, by counting the *unique* tags for each question where the user has a reply, and diving by the unique number of all tags. We may formally define the diversity measure as follows:

Let \\(\mathcal T\\) be the set of all tags and \\(\mathcal T_i\\) be the set of tags in which user \\(i\\) has replied. Then, the diversity measure becomes
$$
\text{Diversity}_i := \frac{\sum_{u\in \mathcal T_i} (u)}{\sum_{v \in \mathcal T}(v)}
$$

The implementation can be summarized as follows:
```
1) Find reply-posts with the original question's corresponding tags and extract tags and reply user ID
2) Map result to a DF with ID and seperated tags
3) Group result by ID and flatten all tag lists
4) Convert the flattened tag list to a set, excluding duplicate tags, and divide the set's length by total number of tags
5) Match user reputation to the newly obtained diversity measure (based on user ID) and compute the correlation coefficient
```

In [0]:
## YOUR IMPLEMENTATION ##
## ___________________ ##
## CREATE DIVERSITY DF ##

def get_tags_list(df: DataFrame) -> Tuple[int, list]:
  # Splits the tag string into a list, also returns the owner user ID
  out_list = [] if df["Tags"] is None else df["Tags"][1:-1].split("><")
  return (df["OwnerUserId"], out_list)

def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B"):
  # SQL Query function for one or two DataFrames 
  df1.createOrReplaceTempView("df1")
  if df2 is not None:
    df2.createOrReplaceTempView("df2")
  out_df = spark.sql(query)
  return out_df

def flatten(df: DataFrame) -> Tuple[int, list]:
  """Flattens lists of
  When we group the data by user ID, we may get an RDD on the form (123, [['A', 'B'], ['C', 'A', 'D'], ...])
  This mapping function uses `itertools.chain` to flatten all arrays into ['A', 'B', 'C', 'A', 'D'], so that
  We can easier determine duplicates and sizes.
  """
  to_flatten = df[1:]
  flattened = []
  for el in to_flatten:
    flattened.append(list(itertools.chain(*el)))
  return (df[0], list(itertools.chain(*flattened)))

q_df = posts_df[posts_df["PostTypeId"] == 1] # Questions
a_df = posts_df[posts_df["PostTypeId"] == 2] # Answers

# Join question-tags to answer post-owner-id since tags are only written in questions-posts.
OUT_0 = run_query2("SELECT df2.Tags, df1.OwnerUserId FROM df1 INNER JOIN df2 ON df1.ParentId = df2.Id", a_df, q_df)
new_rdd = OUT_0.rdd.map(get_tags_list)
diversity_df = new_rdd.groupByKey().mapValues(list).map(flatten).map(lambda df: (df[0], 1/638 * len(set(df[1])))).toDF(["ID", "Diversity"])

In [0]:
## YOUR IMPLEMENTATION ##
## Get Correlation Coefficient ##

corr_df = run_query2("SELECT df1.Diversity, df2.Reputation FROM df1 INNER JOIN df2 ON df1.ID = df2.Id ORDER BY df2.Reputation DESC", diversity_df, users_df)
corr_df.stat.corr("Diversity", "Reputation")

Out[4]: 0.7217677648622942

We see that we get a correlation of about 0.722. Recall that Pearon's correlation coefficient is defined as
$$
\rho_{X,Y} = \frac{\text{Cov}(X,Y)}{\sigma_X\sigma_Y},
$$
measures the linear correlation between the two columns in a DataFrame, and has a value between -1 and 1. If \\(\rho \approx \pm1\\), the correlation between the columns are nearly one-to-one and would follow a pattern \\(X \propto \pm Y\\). In contrast, if the \\(\sigma_{X,Y}\\) is close to zero, there is almost no correlation between the columns.

As such, a strong, positive correlation between diversity and reputation would indicate that users with high reputation writes replies in a large set of tags. Thus, a value of about 0.722 indicates some degree of positive correlation, which suggests that the experts may have more general interests. This is of course not guaranteed, as one can imagine that knowledge seekers asking questions within some topics (e.g. Machine Learning), attach a lot of tags compared to other topics. If this is the case, experts could have a very specialized interest Machine Learning, but based on the correlation have a very diverse tag activity. This downside comes down to how the diversity measure is calculated, since there is naturally correlation between tags within a certain topic as well. In addition, concluding based on the correlation alone may not always be statistically significant, so preforming hypotheses tests would be a better way to determine the question at hand. This is however outside the scope of this course.

In conclusion, the correlation alone seem to indicate that users with higher reputation has a (relatively weak) tendency to reply to questions across multiple tags/topics, but this claim is not rigorously supported.